# Model-Observation Comparison Visualization

This notebook demonstrates the interactive map widget for visualizing model-observation comparisons.
All widget setup, callbacks, and plotting logic has been moved to the `crococamp.viz` module.

In [ ]:
# Option 1: Load pre-existing parquet data (most common use case)
from crococamp.utils.config import read_config
config = read_config("../demo/config.yaml")

import dask.dataframe as dd
ddf = dd.read_parquet(config["parquet_folder"])
ddf.head()

In [ ]:
# Option 2: Generate parquet data using workflow classes (alternative approach)
# This demonstrates the full pipeline from raw data to visualization
# Uncomment and modify the following code if you want to generate data programmatically:

# from crococamp.workflows import WorkflowModelObs
# 
# # Load workflow with configuration
# workflow = WorkflowModelObs.from_config_file("../demo/config.yaml")
# 
# # Run the complete data processing pipeline
# # This will generate parquet files that can then be visualized
# files_processed = workflow.run(trim_obs=True, no_matching=False, parquet_only=True)
# print(f"Processed {files_processed} files")
# 
# # Now load the generated parquet data
# import dask.dataframe as dd
# ddf = dd.read_parquet(workflow.get_config("parquet_folder"))

In [ ]:
# Create and setup the interactive map widget
from crococamp.viz import InteractiveMapWidget, MapConfig

# Optional: customize the map configuration
# config = MapConfig(
#     colormap='viridis',                    # Change colormap
#     plot_title='Custom Analysis Title',   # Change plot title
#     figure_size=(20, 12),                 # Change figure size
#     map_extent=(-180, 180, -90, 90)       # Set custom map extent (lon_min, lon_max, lat_min, lat_max)
# )

# Create the widget with default configuration
widget = InteractiveMapWidget(ddf)  # Add config=config if customizing

# Setup and display the interactive widget
widget.setup()